<a href="https://colab.research.google.com/github/Arjunar92/Hands-On-ML-with-SckitLearn-Keras-TensorFlow/blob/main/02_California_Housing_Price_Prediction_ML_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#This is a code to practice